# Labeling Functions


In [1]:
import re
import glob
import emoji
import fasttext
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from nltk.tokenize import word_tokenize
from nltk import ngrams
from nltk.corpus import stopwords
from sentence_transformers import util

from snorkel.preprocess import preprocessor
from snorkel.labeling import labeling_function
from snorkel.labeling import PandasLFApplier
from snorkel.labeling import LFAnalysis
from snorkel.labeling.model import LabelModel
from sklearn.metrics import classification_report

In [2]:
# defining data paths
prop_data_annotated = "../data/processed/propaganda_annotated.pkl"
gen_data_annotated = "../data/processed/genuine_annotated.pkl"
wanlp_prop_data_path = "../data/raw/task1_train.json"
weakly_labeled_data_path = "../data/processed/train.json"
lfs_analysis_path = "../data/processed/lfs_analysis.json"
lfs_dev_labels_path = "../data/processed/lfs_dev_labels.npy"
lfs_train_labels_path = "../data/processed/lfs_train_labels.npy"

# we need the lf_dev data to report the performance of LFs
# and then remove it with the test data from the unlabaled data
lf_dev_data_path = "../data/processed/lf_dev.json"
test_data_path = "../data/processed/test_data.json"

# defining lexicons paths
proppy_lex_paths = "../data/raw/proppy_lexicons/*.csv"
loaded_lex_path = "../data/raw/loaded-language-lexicons.csv"

# loading pre-trained encoders
fasttext_model_path = "../models/cc.ar.300.bin"

In [3]:
# defining the needed fields by the working labeling functions
# in case of doing the analysis from scratch, comment the variable

# fields = [
#     "tweetid",
#     "user_profile_description",
#     "tweet_text",
#     "is_retweet",
#     "quote_count",
#     "reply_count",
#     "like_count",
#     "retweet_count",
#     "hashtags",
#     "urls",
#     "user_mentions",
#     "text",
#     "emojis",
#     "word_count",
#     "docs",
#     "is_irony",
#     "is_hate",
#     "xlmroberta_label",
# ]

In [4]:
df1 = pd.read_pickle(prop_data_annotated)  # [fields]
df2 = pd.read_pickle(gen_data_annotated)  # [fields]

In [5]:
# the following lines handle date conversion

df1["account_creation_date"] = pd.to_datetime(df1.account_creation_date, unit="ms")
df2["account_creation_date"] = pd.to_datetime(df2.account_creation_date, unit="ns")

In [6]:
# combining the propaganda and genuine users data

unlabeled_data = pd.concat([df1, df2], ignore_index=True)
unlabeled_data = unlabeled_data.sample(frac=1.0).reset_index(drop=True)

In [7]:
unlabeled_data.head(2)

,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,follower_count,following_count,account_creation_date,tweet_text,...,hashtags,urls,user_mentions,text,emojis,word_count,docs,sarcasm,hate,xlmroberta_label
0,661602669614600192,2344913817,ابو سلمان,ghc5r,None,اللهم ارزقني توبتةً نصوحاً قبل الموت #تابعني_ا...,5999.0,5243.0,2014-02-15 00:00:00,💟أَسْتَغْفِرُ اللهَ العَظِيمَ وأتوب إِلَيْهِ💟\...,...,0,0,0,أستغفر الله العظيم وأتوب إليه أستغفر الله العظ...,6,15,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","{'label': 'Non-Sarcasm', 'score': 0.9954224228...","{'label': 'not offensive', 'score': 0.99876511...",{'sequence': 'أستغفر الله العظيم وأتوب إليه أس...
1,1524035570000000000,2183880204,جامعة الباحة,BahaUniversity,الباحة,الحساب الرسمي لـ #جامعة_الباحة | l Al-Baha Uni...,78974.0,32.0,2013-11-09 08:20:37+00:00,#تغطية |وقّع معالي رئيس #جامعة_الباحة أ.د.عبدا...,...,3,4,0,# تغطية|وقع معالي رئيس# جامعة_الباحة أ. د. عبد...,0,35,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","{'label': 'Non-Sarcasm', 'score': 0.9980971217...","{'label': 'not offensive', 'score': 0.99923992...",{'sequence': '# تغطية|وقع معالي رئيس# جامعة_ال...


In [8]:
unlabeled_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196596 entries, 0 to 196595
Data columns (total 25 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   tweetid                   196594 non-null  object 
 1   userid                    196595 non-null  object 
 2   user_display_name         187635 non-null  object 
 3   user_screen_name          187635 non-null  object 
 4   user_reported_location    146198 non-null  object 
 5   user_profile_description  183069 non-null  object 
 6   follower_count            187635 non-null  float64
 7   following_count           187635 non-null  float64
 8   account_creation_date     187635 non-null  object 
 9   tweet_text                196596 non-null  object 
 10  is_retweet                196596 non-null  bool   
 11  quote_count               196591 non-null  float64
 12  reply_count               196592 non-null  float64
 13  like_count                196592 non-null  o

In [9]:
labeled = pd.read_json(lf_dev_data_path)
labeled.head(2)

,tweetid,tweet_text,text,tech,label
0,924924839902793728,RT @Amal_onzi: 🕊💕هُو جنْةبعِيني.,RT : هو جنةبعيني.,None,0
1,1074734231887187970,ر٣ #تركيا_تجاهر_بالمعاصي,ر٣ تركيا تجاهر بالمعاصي,smears - name-calling - loaded language,1


In [10]:
labeled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweetid     500 non-null    int64 
 1   tweet_text  500 non-null    object
 2   text        500 non-null    object
 3   tech        48 non-null     object
 4   label       500 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 23.4+ KB


In [11]:
test = pd.read_json(test_data_path)
test.head(2)

,tweetid,tweet_text,text,tech,label
0,977553193814122498,شاركوا معنا .. في #ساعة_الأرض الليلة \nساعة وا...,شاركوا معنا .. في ساعة الأرض الليلة ساعة واحد ...,None,0
1,1005856990436970497,RT @qtfcjohz: َاللهم طهر قلوبنا من كل ضيق \nوي...,RT : اللهم طهر قلوبنا من كل ضيق ويسر أمورنا في...,None,0


In [12]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 420 entries, 0 to 419
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweetid     420 non-null    int64 
 1   tweet_text  420 non-null    object
 2   text        420 non-null    object
 3   tech        40 non-null     object
 4   label       420 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 19.7+ KB


In [13]:
labeled_data = unlabeled_data[unlabeled_data.tweetid.isin(labeled.tweetid)]
labeled_data.head(2)

,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,follower_count,following_count,account_creation_date,tweet_text,...,hashtags,urls,user_mentions,text,emojis,word_count,docs,sarcasm,hate,xlmroberta_label
30,712221860365205504,2237595005,سـمـو الأمـيـر,K__S__A__511,♥♥الـطـآئـفــے♥♥,الطايف ديرة عشق وذكريات ومقادير اجمل مافي مدن ...,454422.0,98015.0,2013-12-22 00:00:00,-\n\nلا أحد يشعر بأحد\n\nهاهي الشمس تحرق نفسها...,...,1,0,0,- لا أحد يشعر بأحد هاهي الشمس تحرق نفسها من أج...,0,17,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","{'label': 'Non-Sarcasm', 'score': 0.9961795806...","{'label': 'not offensive', 'score': 0.99899071...",{'sequence': '- لا أحد يشعر بأحد هاهي الشمس تح...
636,1064602679287193600,U7gOy6GaZFHMP+S+gYQAgPK3eS2CQTbE1MYwVDkojTA=,U7gOy6GaZFHMP+S+gYQAgPK3eS2CQTbE1MYwVDkojTA=,U7gOy6GaZFHMP+S+gYQAgPK3eS2CQTbE1MYwVDkojTA=,None,سقوط الإنسان ليس فشلا ولكن الفشل أن يبقى حيث سقط,920.0,895.0,2015-11-13 00:00:00,لم يلامس قلبي شئ أعمق من حبك .!\n#غرد_بكلمات_حلوه,...,1,0,0,لم يلامس قلبي شئ أعمق من حبك.!# غرد_بكلمات_حلوه,0,11,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","{'label': 'Non-Sarcasm', 'score': 0.9977585077...","{'label': 'not offensive', 'score': 0.99937540...",{'sequence': 'لم يلامس قلبي شئ أعمق من حبك.!# ...


In [14]:
labeled_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 30 to 196348
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   tweetid                   500 non-null    object 
 1   userid                    500 non-null    object 
 2   user_display_name         500 non-null    object 
 3   user_screen_name          500 non-null    object 
 4   user_reported_location    295 non-null    object 
 5   user_profile_description  462 non-null    object 
 6   follower_count            500 non-null    float64
 7   following_count           500 non-null    float64
 8   account_creation_date     500 non-null    object 
 9   tweet_text                500 non-null    object 
 10  is_retweet                500 non-null    bool   
 11  quote_count               500 non-null    float64
 12  reply_count               500 non-null    float64
 13  like_count                500 non-null    object 
 14  retweet_cou

In [15]:
# removing the labeled tweets from the unlabeled data

unlabeled_data = unlabeled_data[~unlabeled_data.tweetid.isin(labeled_data.tweetid)]
unlabeled_data = unlabeled_data[~unlabeled_data.tweetid.isin(test.tweetid)]
unlabeled_data = unlabeled_data.reset_index(drop=True)

In [16]:
# processing the labaled data tweets

from arabert.preprocess import ArabertPreprocessor

processing_model = "aubmindlab/bert-base-arabertv2"
arabert_prep = ArabertPreprocessor(model_name=processing_model)


def process_text(text):
    """Process tweet text by removing links, mentions, and hashtags symbol."""
    clean_text = arabert_prep.preprocess(text)
    clean_text = arabert_prep.unpreprocess(clean_text)
    clean_text = clean_text.replace("[رابط]", "")
    clean_text = clean_text.replace("[مستخدم]", "")
    clean_text = clean_text.replace("RT", "")
    clean_text = clean_text.replace(":", "")
    return clean_text.strip()

[2023-11-14 15:35:42,854 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


In [17]:
labeled_data["text"] = labeled_data.text.apply(process_text)
labeled_data = labeled_data.reset_index(drop=True)
labels = [labeled[labeled.tweetid == i].label.values[0] for i in labeled_data.tweetid]
labeled_data["label"] = labels

In [18]:
# making sure the dataset doesn't contain any null values

subset = ["tweetid", "text", "quote_count", "xlmroberta_label"]
unlabeled_data = unlabeled_data.dropna(subset=subset)
unlabeled_data = unlabeled_data.reset_index(drop=True)

In [19]:
unlabeled_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195671 entries, 0 to 195670
Data columns (total 25 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   tweetid                   195671 non-null  object 
 1   userid                    195671 non-null  object 
 2   user_display_name         186715 non-null  object 
 3   user_screen_name          186715 non-null  object 
 4   user_reported_location    145634 non-null  object 
 5   user_profile_description  182217 non-null  object 
 6   follower_count            186715 non-null  float64
 7   following_count           186715 non-null  float64
 8   account_creation_date     186715 non-null  object 
 9   tweet_text                195671 non-null  object 
 10  is_retweet                195671 non-null  bool   
 11  quote_count               195671 non-null  float64
 12  reply_count               195671 non-null  float64
 13  like_count                195671 non-null  o

In [20]:
labeled_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   tweetid                   500 non-null    object 
 1   userid                    500 non-null    object 
 2   user_display_name         500 non-null    object 
 3   user_screen_name          500 non-null    object 
 4   user_reported_location    295 non-null    object 
 5   user_profile_description  462 non-null    object 
 6   follower_count            500 non-null    float64
 7   following_count           500 non-null    float64
 8   account_creation_date     500 non-null    object 
 9   tweet_text                500 non-null    object 
 10  is_retweet                500 non-null    bool   
 11  quote_count               500 non-null    float64
 12  reply_count               500 non-null    float64
 13  like_count                500 non-null    object 
 14  retweet_co

### User LFs


In [21]:
prop = 1
gen = 0
ab = -1

In [22]:
@labeling_function()
def missing_bio(example):
    """Label all tweets of a user as propaganda if they don't have a bio."""
    if pd.isna(example.user_profile_description):
        return prop
    else:
        return ab

In [23]:
@labeling_function()
def missing_loc(example):
    """Label all tweets of a user as propaganda if they don't have a location."""
    if pd.isna(example.user_reported_location):
        return prop
    else:
        return ab

In [24]:
@labeling_function()
def created_2018_2019(example):
    """Label all tweets of a user as propaganda if account is created 2018 or 2019."""
    if example.account_creation_date.year in [2018, 2019]:
        return prop
    else:
        return ab

In [25]:
@labeling_function()
def created_2011_2012(example):
    """Label all tweets of a user as transparent if account is created 2011 or 2012."""
    if example.account_creation_date.year in [2011, 2012]:
        return gen
    else:
        return ab

In [26]:
@preprocessor(memoize=False)
def tokenize_bio(example):
    """Tokenize text in the bio."""
    if not pd.isna(example.user_profile_description):
        example.bio_tokens = word_tokenize(example.user_profile_description)
    else:
        example.bio_tokens = None
    return example

In [27]:
@labeling_function(pre=[tokenize_bio])
def bio_keywords(example):
    """Label all tweets of a user as transparent if bio contains certain lexicons."""
    keys = ["الحساب", "الرسمي", "عضو", "رئيس", "كاتب", "إدارة"]
    if example.bio_tokens is not None:
        if any(np.in1d(keys, example.bio_tokens)):
            return gen
        else:
            return ab
    else:
        return ab

In [28]:
@labeling_function()
def follow_ratio_prop(example):
    """Label all tweets of a user as transparent if 0.8 <= follow ratio <= 1.2"""
    num_followers = example.follower_count
    num_following = example.following_count
    ratio = num_followers / num_following if num_following != 0 else 0
    if 0.8 <= ratio <= 1.2:
        return prop
    else:
        return ab

In [29]:
@labeling_function()
def follow_ratio_gen(example):
    """Label all tweets of a user as transparent if follow ratio <= 0.2"""
    num_followers = example.follower_count
    num_following = example.following_count
    ratio = num_followers / num_following if num_following != 0 else 0
    if 0.0 <= ratio <= 0.2:
        return gen
    else:
        return ab

### Tweet LFs


In [30]:
@labeling_function()
def contain_url(example):
    """Label tweet as transparent if it contains a URL."""
    if example.urls > 0:
        return gen
    else:
        return ab

In [31]:
@labeling_function()
def contain_mention(example):
    """Label tweet as propaganda if it contains a mention."""
    if example.user_mentions > 0:
        return prop
    else:
        return ab

In [32]:
@labeling_function()
def labeling_sarcasm(example):
    """Label tweet as propaganda if it contains the name calling tech (sarcasm)."""
    if len(example.docs.entities) and example.sarcasm["label"] == "Sarcasm":
        return prop
    else:
        return ab

In [33]:
@labeling_function()
def labeling_hate(example):
    """Label tweet as propaganda if it contains the name calling tech (hate)."""
    if len(example.docs.entities) and example.hate["label"] == "offensive":
        return prop
    else:
        return ab

In [34]:
@labeling_function()
def contain_ent(example):
    """Label tweet as propaganda if it contains an entity."""
    if len(example.docs.entities):
        return prop
    else:
        return ab

In [35]:
@labeling_function()
def ent_free(example):
    """Label tweet as transparent if it doesn't contain any entities."""
    if len(example.docs.entities):
        return ab
    else:
        return gen

In [36]:
@labeling_function()
def contain_question(example):
    """Label tweet as propaganda if it contains an question."""
    for w in example.docs.sentences[0].words:
        if w.upos == "AUX":
            return prop
    return ab

In [37]:
# loading the manually crafted loaded tokens.
# extracted only from the 500 tweets used for labeling functions development.

loaded_lexicons = pd.read_csv(loaded_lex_path)["loaded-language"].to_list()
loaded_lexicons[:5]

['جاهر', 'جهر', 'تجاهر بالمعاصي', 'مجاهرة بالمعاصي', 'تجاهر بالمعاصى']

In [38]:
# loading the proppy lexicons introduced in https://arxiv.org/abs/1912.06810

# proppy_lexicons = []
# for file in glob.glob(proppy_lex_paths):
#     with open(file, encoding="utf-8") as f:
#         proppy_lexicons.extend(f.readlines())

proppy_lexicons = pd.read_csv(glob.glob(proppy_lex_paths)[0])
proppy_lexicons.head(2)

,factives,hedges,implicatives,report_verbs,bias,negative_words,positive_words,negative_colloquial_words,positive_colloquial_words
0,عارف,واضح,يدير,يتهم,إسقاط,أبو وجهين,فالح,مزوره,متحمس
1,يدري,فيما يبدو,يتذكر,اتهم,إجهاض,بوجهين,شاطر,مغترب,حماس


In [39]:
# removing diacritization from proppy lexicons

# pattern = r"[\u0617-\u061A\u064B-\u0652]"
# proppy_lexicons = [re.sub(pattern, "", term.strip()) for term in proppy_lexicons]

In [40]:
proppy_lexicons = proppy_lexicons.to_dict(orient="list")

In [41]:
proppy_lexicons = {
    k: [w for w in v if not pd.isna(w)] for k, v in proppy_lexicons.items()
}

In [42]:
print("Number of words in each proppy lexicon type ...")
print([len(v) for v in proppy_lexicons.values()])

Number of words in each proppy lexicon type ...
[42, 109, 43, 212, 578, 4630, 1950, 1468, 914]


In [43]:
# combining all the proppy lexicons

proppy = []
for plex in proppy_lexicons:
    proppy.extend(plex)

In [44]:
# we are removing diacritization from stanza lemmas


@preprocessor(memoize=False)
def tokenize_tweet(example):
    """Tokenize and lemmatize text in tweets."""
    if len(example.docs.sentences):
        example.tweet_tokens = [w.text for w in example.docs.sentences[0].words]
        example.tweet_lemmas = [
            re.sub(pattern, "", w.lemma) for w in example.docs.sentences[0].words
        ]
    else:
        example.tweet_tokens = []
        example.tweet_lemmas = []
    return example

In [45]:
@preprocessor(pre=[tokenize_tweet], memoize=False)
def bigram_tweet(example):
    """Create bigrams of tweet text's tokens and lemmas."""
    example.bigram_tokens = [
        " ".join(gram) for gram in ngrams(example.tweet_tokens, 2) if len(gram)
    ]
    example.bigram_lemmas = [
        " ".join(gram) for gram in ngrams(example.tweet_lemmas, 2) if len(gram)
    ]
    return example

In [46]:
@labeling_function(pre=[bigram_tweet])
def loaded_language(example):
    """Label tweet as propaganda if it contains any of the loaded lexicons."""
    tweet_ngrams = example.tweet_tokens + example.tweet_lemmas
    tweet_ngrams += example.bigram_tokens + example.bigram_lemmas
    if sum(np.in1d(tweet_ngrams, loaded_lexicons)) >= 1:
        return prop
    else:
        return ab

In [47]:
@labeling_function(pre=[bigram_tweet])  # [bigram_tweet]
def genuine_language(example):
    """Label tweet as transparent if it doesn't contain any of the loaded lexicons."""
    tweet_ngrams = example.tweet_tokens + example.tweet_lemmas
    tweet_ngrams += example.bigram_tokens + example.bigram_lemmas
    if sum(np.in1d(tweet_ngrams, loaded_lexicons)) == 0:
        return gen
    else:
        return ab

In [48]:
@labeling_function(pre=[bigram_tweet])
def loaded_proppy(example):
    """Label tweet as propaganda if it contains any of the proppy lexicons."""
    tweet_ngrams = example.tweet_tokens + example.tweet_lemmas
    tweet_ngrams += example.bigram_tokens + example.bigram_lemmas
    if sum(np.in1d(tweet_ngrams, proppy)) >= 1:
        return prop
    else:
        return ab

In [49]:
@labeling_function(pre=[bigram_tweet])
def genuine_proppy(example):
    """Label tweet as transparent if it doesn't contain any of the proppy lexicons."""
    tweet_ngrams = example.tweet_tokens + example.tweet_lemmas
    tweet_ngrams += example.bigram_tokens + example.bigram_lemmas
    if sum(np.in1d(tweet_ngrams, proppy)) == 0:
        return gen
    else:
        return ab

The following commented codes are to split each proppy lexicon type.

In [50]:
name_space = {}
proppy_lfs = []

In [51]:
for name in proppy_lexicons.keys():
    code = f"""
@labeling_function(pre=[bigram_tweet])
def loaded_proppy_{name}(example):
    \"""Label tweet as propaganda if it contains any of the proppy lexicons.\"""
    tweet_ngrams = example.tweet_tokens + example.tweet_lemmas
    tweet_ngrams += example.bigram_tokens + example.bigram_lemmas
    if sum(np.in1d(tweet_ngrams, proppy_lexicons["{name}"])) >= 1:
        return prop
    else:
        return ab
    """
    code = compile(code, "<string>", "exec")
    exec(code, globals(), name_space)
    proppy_lfs.append(name_space[f"loaded_proppy_{name}"])

In [52]:
for name in proppy_lexicons.keys():
    code = f"""
@labeling_function(pre=[bigram_tweet])
def genuine_proppy_{name}(example):
    \"""Label tweet as transparent if it doesn't contain any of the proppy lexicons.\"""
    tweet_ngrams = example.tweet_tokens + example.tweet_lemmas
    tweet_ngrams += example.bigram_tokens + example.bigram_lemmas
    if sum(np.in1d(tweet_ngrams, proppy_lexicons["{name}"])) == 0:
        return gen
    else:
        return ab
    """
    code = compile(code, "<string>", "exec")
    exec(code, globals(), name_space)
    proppy_lfs.append(name_space[f"genuine_proppy_{name}"])

In [53]:
len(proppy_lfs)

18

In [54]:
@labeling_function()
def loaded_hate(example):
    """Label tweet as propaganda if it contains hate speech (loaded language)."""
    if example.hate["label"] == "offensive":
        return prop
    else:
        return ab

In [55]:
@labeling_function()
def loaded_sarcasm(example):
    """Label tweet as propaganda if it contains sarcasm (loaded language)."""
    if example.sarcasm["label"] == "Sarcasm":
        return prop
    else:
        return ab

In [56]:
@labeling_function()
def gen_hate(example):
    """Label tweet as transparent if it doesn't contain hate speech (loaded language)."""
    if example.hate["label"] == "not offensive":
        return gen
    else:
        return ab

In [57]:
@labeling_function()
def gen_sarcasm(example):
    """Label tweet as transparent if it doesn't contain sarcasm (loaded language)."""
    if example.sarcasm["label"] == "Non-Sarcasm":
        return gen
    else:
        return ab

In [58]:
# flag_waving

flag_engine = re.compile(r"Number=\w+")


@labeling_function()
def flag_wave(example):
    """Label tweet as propaganda if it contains plural pronouns (flag-waving)."""
    for w in example.docs.sentences[0].words:
        if (
            w.upos == "PRON"
            and flag_engine.findall(w.feats)[0].split("=")[-1] == "Plur"
        ):
            return prop
    return ab

In [59]:
# loading the Arabic NLTK stop words.
arabic_stop_words = stopwords.words("arabic")

# removing diacritization from stop words
pattern = r"[\u0617-\u061A\u064B-\u0652]"
arabic_stop_words = [re.sub(pattern, "", w) for w in arabic_stop_words]

In [60]:
@labeling_function(pre=[tokenize_tweet])
def repetition(example):
    """Label tweet as propaganda if it has at least one repeated token."""
    tokens = [word for word in example.tweet_tokens if word not in arabic_stop_words]
    tokens = pd.Series(tokens)
    if tokens.value_counts().max() >= 2:
        return prop
    else:
        return ab

In [61]:
# loading the WANLP propaganda dataset for distant supervision
# dataset source: https://gitlab.com/araieval/propaganda-detection


wanlp_train = pd.read_json(wanlp_prop_data_path)

label_processing = lambda x: 0 if "no technique" in x else 1
wanlp_train["labels"] = wanlp_train["labels"].apply(label_processing)

wanlp_train.head(2)

,id,text,labels
0,1358824915483435008,"#بي_بي_سي_ترندينغ: النساء ""تثرثر كثيرا"" رئيس أ...",0
1,1389927866356412416,"""ده مش معتقل ده أحسن من اللوكاندة"".. جدل وسخري...",1


In [62]:
# def clean_text(text):
#     """Process text and remove links, symbols, and diacritization."""
#     # links
#     clean_text = re.sub(r"http\S+|t\.co/\S+", "", text)
#     # mentions
#     clean_text = re.sub(r"@\w+", "", clean_text)
#     # hashtags
#     clean_text = re.sub(r"#", "", clean_text)
#     clean_text = re.sub(r"_", " ", clean_text)
#     # tashqeel - from @bakriano
#     clean_text = re.sub(r"[\u0617-\u061A\u064B-\u0652]", "", clean_text)
#     # emojis
#     clean_text = emoji.replace_emoji(clean_text, replace="")
#     # remove new lines and normalize white spaces
#     clean_text = re.sub(r"\s+", " ", clean_text)
#     return clean_text.replace("RT :", "").strip()


wanlp_train["text"] = wanlp_train["text"].apply(process_text)

In [63]:
encoder = fasttext.load_model(fasttext_model_path)

wanlp_hidden_states = []
for tweet in tqdm(wanlp_train.text.values, total=len(wanlp_train)):
    vec = encoder.get_sentence_vector(tweet)
    wanlp_hidden_states.append(list(vec))

  0%|          | 0/504 [00:00<?, ?it/s]

In [64]:
wanlp_hidden_states = np.array(wanlp_hidden_states)
wanlp_hidden_states.shape

(504, 300)

In [65]:
@preprocessor()
def get_sim_scores(example):
    """Get similarity score between tweet and WANLP propaganda tweets."""
    tweet_vec = encoder.get_sentence_vector(example.text)
    sim_scores = util.cos_sim(tweet_vec, wanlp_hidden_states)
    example.sim_scores = sim_scores
    return example

In [66]:
# The included scores are fine-tuned.


@labeling_function(pre=[get_sim_scores])
def distant_supervision_prop(example):
    """Label tweet as propaganda based on its most similar WANLP example."""
    sim_scores = example.sim_scores
    most_sim = sim_scores.argmax(dim=-1).item()
    if (
        wanlp_train.labels.values[most_sim] == 1
        and sim_scores[-1][most_sim].item() >= 0.80
    ):
        return prop
    else:
        return ab

In [67]:
# The included scores are fine-tuned.


@labeling_function(pre=[get_sim_scores])
def distant_supervision_gen(example):
    """Label tweet as transparent based on its most similar WANLP example."""
    sim_scores = example.sim_scores
    most_sim = sim_scores.argmax(dim=-1).item()
    if (
        wanlp_train.labels.values[most_sim] == 0
        and sim_scores[-1][most_sim].item() >= 0.60
    ):
        return gen
    else:
        return ab

In [68]:
@labeling_function()
def slogans(example):
    """Label tweet as propaganda if it has any of the slogans form."""
    matches = re.findall(r"لا ل\w+", example.text)
    matches += re.findall(r"نعم ل\w+", example.text)
    matches += re.findall(r"لا بديل", example.text)
    if len(matches):
        return prop
    else:
        return ab

In [69]:
# defining the manually extracted hated organizations and entities.

hitlerum = [
    "هتلر",
    "البغدادي",
    "اوردوغان",
    "قطر",
    "داعش",
    "حوثي",
    "تركيا",
    "الشيعة",
    "إيران",
    "ايران",
    "اخونجي",
    "اخوان",
    "إخوان",
    "إخوانجي",
    "أوردوغان",
    "الحوثي",
    "الحوثيين",
    "ستالين",
    "الإخوان",
    "الاخوان",
    "إرهابي",
    "الإرهابيين",
    "متطرف",
    "المتطرفين",
    "شيعي",
]

In [70]:
@labeling_function(pre=[bigram_tweet])
def reductio(example):
    """Label tweet as propaganda if it contains the Reductio Ad Hitlerum tech."""
    tweet_ngrams = example.tweet_tokens + example.tweet_lemmas
    tweet_ngrams += example.bigram_tokens + example.bigram_lemmas
    if any(np.in1d(tweet_ngrams, hitlerum)):
        return prop
    else:
        return ab

In [71]:
@labeling_function()
def exaggeration(example):
    """Label tweet as propaganda if has the "أفعل" preference form."""
    if not len(example.docs.sentences):
        return ab
    for w in example.docs.sentences[0].words:
        if w.lemma.startswith("أ") and w.upos == "ADJ":
            return prop
    return ab

In [72]:
@labeling_function()
def pronouns(example):
    """Label tweet as propaganda if has at least one pronoun."""
    pro_nouns = []
    for word in example.docs.sentences[0].words:
        if word.upos == "PRON":
            pro_nouns.append(word.text)
    if len(pro_nouns) >= 1:
        return prop
    else:
        return ab

In [73]:
# defining the XLM-RoBERTa Zero-Shot model classes
candidate_labels = ["transparent", "propaganda"]
candidate_labels

['transparent', 'propaganda']

In [74]:
# The included scores are fine-tuned.


@labeling_function()
def xlmroberta_prop(example):
    """Label tweet as propaganda based on zero-shot model."""
    if pd.isna(example.xlmroberta_label):
        return ab
    if example.xlmroberta_label["scores"][0] >= 0.90:
        if example.xlmroberta_label["labels"][0] == candidate_labels[-1]:
            return prop
        else:
            return ab
    else:
        return ab

In [75]:
# The included scores are fine-tuned.


@labeling_function()
def xlmroberta_gen(example):
    """Label tweet as transparent based on zero-shot model."""
    if pd.isna(example.xlmroberta_label):
        return ab
    if example.xlmroberta_label["scores"][0] >= 0.75:
        if example.xlmroberta_label["labels"][0] == candidate_labels[0]:
            return gen
        else:
            return ab
    else:
        return ab

In [76]:
lfs = [
    missing_bio,
    missing_loc,
    created_2018_2019,
    created_2011_2012,
    bio_keywords,
    follow_ratio_prop,
    follow_ratio_gen,
    contain_url,
    contain_mention,
    labeling_sarcasm,
    labeling_hate,
    contain_ent,
    ent_free,
    contain_question,
    loaded_language,
    genuine_language,
    loaded_proppy,
    genuine_proppy,
    loaded_hate,
    loaded_sarcasm,
    gen_hate,
    gen_sarcasm,
    flag_wave,
    repetition,
    distant_supervision_prop,
    distant_supervision_gen,
    slogans,
    reductio,
    exaggeration,
    pronouns,
    xlmroberta_prop,
    xlmroberta_gen,
]

lfs += proppy_lfs  # uncomment this line in case of splitted proppy lex

print(f"We have {len(lfs)} LFs used.")

We have 50 LFs used.


In [126]:
applier = PandasLFApplier(lfs)

In [127]:
L_dev = applier.apply(labeled_data)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:23<00:00, 21.73it/s]


In [128]:
results = LFAnalysis(L_dev, lfs).lf_summary(labeled_data.label.values)
results.to_json(lfs_analysis_path)
results

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
missing_bio,0,[1],0.076,0.076,0.076,3,35,0.078947
missing_loc,1,[1],0.410,0.410,0.410,20,185,0.097561
created_2018_2019,2,[1],0.084,0.084,0.084,3,39,0.071429
created_2011_2012,3,[0],0.288,0.288,0.286,127,17,0.881944
bio_keywords,4,[0],0.066,0.066,0.066,32,1,0.969697
follow_ratio_prop,5,[1],0.216,0.216,0.216,8,100,0.074074
follow_ratio_gen,6,[0],0.004,0.004,0.004,2,0,1.000000
contain_url,7,[0],0.044,0.044,0.044,21,1,0.954545
contain_mention,8,[1],0.652,0.652,0.652,29,297,0.088957
labeling_sarcasm,9,[1],0.006,0.006,0.006,2,1,0.666667


In [129]:
L_train = applier.apply(unlabeled_data)
LFAnalysis(L_train, lfs).lf_summary()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 195671/195671 [3:24:17<00:00, 15.96it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
missing_bio,0,[1],0.068758,0.068758,0.068758
missing_loc,1,[1],0.255720,0.255720,0.255720
created_2018_2019,2,[1],0.048530,0.048530,0.048530
created_2011_2012,3,[0],0.371465,0.371465,0.368997
bio_keywords,4,[0],0.326385,0.326385,0.324918
follow_ratio_prop,5,[1],0.057530,0.057530,0.057530
follow_ratio_gen,6,[0],0.002054,0.002054,0.002019
contain_url,7,[0],0.461223,0.461223,0.458325
contain_mention,8,[1],0.478221,0.478221,0.478221
labeling_sarcasm,9,[1],0.006307,0.006307,0.006307


In [130]:
L_dev_ids = L_dev.copy()
L_train_ids = L_train.copy()
L_dev_ids = np.append(L_dev_ids, labeled_data.tweetid.to_numpy().reshape(-1, 1), axis=1)
L_train_ids = np.append(
    L_train_ids, unlabeled_data.tweetid.to_numpy().reshape(-1, 1), axis=1
)
L_dev_ids.shape, L_train_ids.shape

((500, 51), (195671, 51))

In [131]:
np.save(lfs_dev_labels_path, L_dev_ids)
np.save(lfs_train_labels_path, L_train_ids)

In [77]:
# loading the saved data

results = pd.read_json(lfs_analysis_path)
L_train = np.load(lfs_train_labels_path, allow_pickle=True)
L_dev = np.load(lfs_dev_labels_path, allow_pickle=True)

In [78]:
results.head(2)

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
missing_bio,0,[1],0.076,0.076,0.076,3,35,0.078947
missing_loc,1,[1],0.410,0.410,0.410,20,185,0.097561


In [79]:
L_dev.shape, L_train.shape

((500, 51), (195671, 51))

In [80]:
L_train[:2, -1]

array([832915646832840706, 1356667243808055298], dtype=object)

In [81]:
unlabeled_data.head(2)

,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,follower_count,following_count,account_creation_date,tweet_text,...,hashtags,urls,user_mentions,text,emojis,word_count,docs,sarcasm,hate,xlmroberta_label
0,661602669614600192,2344913817,ابو سلمان,ghc5r,None,اللهم ارزقني توبتةً نصوحاً قبل الموت #تابعني_ا...,5999.0,5243.0,2014-02-15 00:00:00,💟أَسْتَغْفِرُ اللهَ العَظِيمَ وأتوب إِلَيْهِ💟\...,...,0,0,0,أستغفر الله العظيم وأتوب إليه أستغفر الله العظ...,6,15,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","{'label': 'Non-Sarcasm', 'score': 0.9954224228...","{'label': 'not offensive', 'score': 0.99876511...",{'sequence': 'أستغفر الله العظيم وأتوب إليه أس...
1,1524035570000000000,2183880204,جامعة الباحة,BahaUniversity,الباحة,الحساب الرسمي لـ #جامعة_الباحة | l Al-Baha Uni...,78974.0,32.0,2013-11-09 08:20:37+00:00,#تغطية |وقّع معالي رئيس #جامعة_الباحة أ.د.عبدا...,...,3,4,0,# تغطية|وقع معالي رئيس# جامعة_الباحة أ. د. عبد...,0,35,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","{'label': 'Non-Sarcasm', 'score': 0.9980971217...","{'label': 'not offensive', 'score': 0.99923992...",{'sequence': '# تغطية|وقع معالي رئيس# جامعة_ال...


In [82]:
unlabeled_data = (
    unlabeled_data.set_index(unlabeled_data.tweetid)
    .loc[L_train[:, -1]]
    .reset_index(drop=True)
)
unlabeled_data.head(2)

,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,follower_count,following_count,account_creation_date,tweet_text,...,hashtags,urls,user_mentions,text,emojis,word_count,docs,sarcasm,hate,xlmroberta_label
0,832915646832840706,608481947,راعية الخيمه🦅 وتسذاا🐪🌴🎪,alflak2007,المدينة المنورة🇸🇦,﷽{وَإِن قِيلَ لَكُمُ ارْجِعُوا فَارْجِعُوا هُو...,257869.0,231522.0,2012-06-14 00:00:00,RT @Mn_m81: #الاميرة_شيخة_مطرانها_22k_محب\n┊🌸م...,...,2,0,2,# الأميرة_شيخة_مطرانها_22k_محب متميزه راقيه مب...,20,9,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","{'label': 'Non-Sarcasm', 'score': 0.9973012804...","{'label': 'not offensive', 'score': 0.99903464...",{'sequence': '# الأميرة_شيخة_مطرانها_22k_محب م...
1,1356667243808055298,732626805782392832,أمل عبدالعزيزالهزاني,Alhazzani_Amal,"Riyadh, Kingdom of Saudi Arabia",أ. د. في جامعة الملك سعود/كلية العلوم- وراثة ج...,39681.0,2561.0,2016-05-17 17:40:16+00:00,@BaderAlJunaydil شكرا جزيلا بدر على ارفاق جزء ...,...,0,0,1,شكرا جزيلا بدر على ارفاق جزء من المقال مع صورة...,1,15,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","{'label': 'Non-Sarcasm', 'score': 0.9977416992...","{'label': 'not offensive', 'score': 0.99948918...",{'sequence': 'شكرا جزيلا بدر على ارفاق جزء من ...


In [83]:
labeled_data = (
    labeled_data.set_index(labeled_data.tweetid)
    .loc[L_dev[:, -1]]
    .reset_index(drop=True)
)
labeled_data.head(2)

,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,follower_count,following_count,account_creation_date,tweet_text,...,urls,user_mentions,text,emojis,word_count,docs,sarcasm,hate,xlmroberta_label,label
0,756959210005463040,736422112785793024,🇸🇦عالي الشرافاء🇸🇦,Zs100s,"الرياض, المملكة العربية السعود",‏التغريدات التي تمثلني فقط💙 بالمفضله💙، واسأل ا...,85130.0,73332.0,2016-05-28 00:00:00,RT @Aldoserymy: اللهم \nارحم كل ميت واشفِ كل م...,...,0,1,اللهم ارحم كل ميت واشف كل مريض واحفظ كل مسلم و...,0,23,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","{'label': 'Non-Sarcasm', 'score': 0.9950862526...","{'label': 'not offensive', 'score': 0.99965584...",{'sequence': 'اللهم ارحم كل ميت واشف كل مريض و...,0
1,554575287208255488,1320537474,زياد اليامي 🎗📸🇸🇦.,zyaeddb,الرياض،نجران,"معلم في المستقبل "" مكافح"" مصور جريدة الرياض وا...",30503.0,2166.0,2013-04-01 00:00:00,"""\n\nسُبحانك ربي عدد ما سَبحك المُسبحون،\nوأست...",...,0,0,""" سبحانك ربي عدد ما سبحك المسبحون، وأستغفرك رب...",0,15,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","{'label': 'Non-Sarcasm', 'score': 0.9968816041...","{'label': 'not offensive', 'score': 0.99916219...","{'sequence': '"" سبحانك ربي عدد ما سبحك المسبحو...",0


In [84]:
assert len(unlabeled_data) == len(L_train)
assert len(labeled_data) == len(L_dev)

In [85]:
L_train = L_train[:, :-1].copy()
L_dev = L_dev[:, :-1].copy()

In [86]:
L_dev.shape, L_train.shape

((500, 50), (195671, 50))

In [87]:
L_train = L_train.astype(np.int32)
L_dev = L_dev.astype(np.int32)

In [88]:
results

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
missing_bio,0,[1],0.076,0.076,0.076,3,35,0.078947
missing_loc,1,[1],0.410,0.410,0.410,20,185,0.097561
created_2018_2019,2,[1],0.084,0.084,0.084,3,39,0.071429
created_2011_2012,3,[0],0.288,0.288,0.286,127,17,0.881944
bio_keywords,4,[0],0.066,0.066,0.066,32,1,0.969697
follow_ratio_prop,5,[1],0.216,0.216,0.216,8,100,0.074074
follow_ratio_gen,6,[0],0.004,0.004,0.004,2,0,1.000000
contain_url,7,[0],0.044,0.044,0.044,21,1,0.954545
contain_mention,8,[1],0.652,0.652,0.652,29,297,0.088957
labeling_sarcasm,9,[1],0.006,0.006,0.006,2,1,0.666667


In [89]:
# selecting the LFs that maximize the performance of the label model
flag = ~np.array([results["Emp. Acc."] < 0.2, results["Coverage"] >= 0.5]).any(axis=0)
inds = results[flag].j.to_list()
# remove user signals
inds.remove(3)
inds.remove(6)
# append tweet signals
inds.append(15)
len(inds)

16

In [90]:
# displaying the names of the LFs chosen
results.iloc[inds, :].index.to_list()

['bio_keywords',
 'contain_url',
 'labeling_sarcasm',
 'contain_question',
 'loaded_language',
 'loaded_hate',
 'loaded_sarcasm',
 'distant_supervision_prop',
 'distant_supervision_gen',
 'slogans',
 'reductio',
 'xlmroberta_prop',
 'xlmroberta_gen',
 'loaded_proppy_factives',
 'loaded_proppy_implicatives',
 'genuine_language']

In [91]:
L_train_ = L_train[:, inds].copy()
L_dev_ = L_dev[:, inds].copy()

In [92]:
LFAnalysis(L_dev_, np.array(lfs)[inds]).lf_summary(labeled_data.label.values)

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
bio_keywords,0,[0],0.066,0.066,0.016,32,1,0.969697
contain_url,1,[0],0.044,0.044,0.008,21,1,0.954545
labeling_sarcasm,2,[1],0.006,0.006,0.006,2,1,0.666667
contain_question,3,[1],0.064,0.064,0.058,8,24,0.250000
loaded_language,4,[1],0.162,0.118,0.104,30,51,0.370370
loaded_hate,5,[1],0.032,0.032,0.030,4,12,0.250000
loaded_sarcasm,6,[1],0.008,0.008,0.008,3,1,0.750000
distant_supervision_prop,7,[1],0.008,0.008,0.008,3,1,0.750000
distant_supervision_gen,8,[0],0.240,0.240,0.066,111,9,0.925000
slogans,9,[1],0.006,0.006,0.006,1,2,0.333333


In [93]:
# estimating the class balance in the unlabeled data from the labeled.

w = labeled_data.label.value_counts(normalize=True).to_list()
w

[0.904, 0.096]

In [94]:
# tuning the L2 regularization parameter to maximize the performance of the label model
# we maximize the precision as it is need to surpass 50% for the noise-aware loss

best_score = 0
best_model = None
l2_values = np.arange(0.0, 0.1, 0.01)


metric = "f1"
for l2 in tqdm(l2_values, total=len(l2_values)):
    label_model = LabelModel(cardinality=2, verbose=False, device="cuda")
    label_model.fit(
        L_train=L_train_,
        n_epochs=2500,
        l2=l2,
        lr=0.001,
        seed=42,
        class_balance=None,
        progress_bar=False,
        lr_scheduler="constant",
        optimizer="sgd",
        lr_scheduler_config={"warmup_percentage": 0.0, "warmup_unit": "epochs"},
    )
    score = label_model.score(L_dev_, labeled_data.label, metrics=[metric])
    if score[metric] >= best_score:
        best_score = score[metric]
        best_model = label_model

  0%|          | 0/10 [00:00<?, ?it/s]

In [95]:
# displaying the configs that led to the best label model
best_model.train_config

TrainConfig(n_epochs=2500, lr=0.001, l2=0.04, optimizer='sgd', optimizer_config=OptimizerConfig(sgd_config=SGDOptimizerConfig(momentum=0.9), adam_config=AdamOptimizerConfig(amsgrad=False, betas=(0.9, 0.999)), adamax_config=AdamaxOptimizerConfig(betas=(0.9, 0.999), eps=1e-08)), lr_scheduler='constant', lr_scheduler_config=LRSchedulerConfig(warmup_steps=0, warmup_unit='epochs', warmup_percentage=0.0, min_lr=0.0, exponential_config=ExponentialLRSchedulerConfig(gamma=0.9), step_config=StepLRSchedulerConfig(gamma=0.9, step_size=5)), prec_init=0.7, seed=42, log_freq=10, mu_eps=None)

Reporting the performance of the label model


In [96]:
best_model.score(
    L_dev_,
    labeled_data.label,
    tie_break_policy="abstain",
    metrics=["accuracy", "precision", "recall", "f1", "f1_macro", "f1_micro"],
)

{'accuracy': 0.866,
 'precision': 0.39080459770114945,
 'recall': 0.7083333333333334,
 'f1': 0.5037037037037037,
 'f1_macro': 0.7131235281524297,
 'f1_micro': 0.866}

In [97]:
y_true = labeled_data.label
y_pred = best_model.predict(L_dev_)
y_true = y_true[y_pred != -1]
y_pred = y_pred[y_pred != -1]

In [98]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.88      0.92       452
           1       0.39      0.71      0.50        48

    accuracy                           0.87       500
   macro avg       0.68      0.80      0.71       500
weighted avg       0.91      0.87      0.88       500



Saving the weakly labeled dataset


In [99]:
# labeled_data["snorkel"] = best_model.predict(L_dev_)

In [100]:
# saving the datasets
weak_probs = best_model.predict_proba(L_train_)
weak_preds = best_model.predict(L_train_)
unlabeled_data["label"] = weak_preds
unlabeled_data["probs"] = list(weak_probs)
unlabeled_data = unlabeled_data[weak_preds != -1]

In [101]:
unlabeled_data[["tweetid", "text", "label", "probs"]].to_json(weakly_labeled_data_path)